
# MongoDB desde 0.  Fichero 01.
Quiero tener un curso de python, mongoBD y SQL basado en proyectos.  
En esta serie de artículos escritos sobre Jupyter (.ipynd) será mi cuaderno de bitácora.  
En general el proyecto es correr un contenedor Docker que permita pasar todos los clientes de la base de datos de MongoDB antígua a la nueva versión. Para ello usaré el VSC (Visual Studio Code) como IDE y el Python como lenguaje de programación.  
Partiré de conocimientos básicos de Python aprendidos en la serie *xx LSP aprendo*.  

### La base
Voy a construir un código con VSC que correrá realmente en un contenedor de Docker. De ahí supongo que lo primero que tengo que hacer es encontrar un contenedor de Docker que ya venga con un Python instalado.  

Luego miraré como enlazar el VSC a dicho Docker.  

Por último empezaré a crear código que se ejecute en el contenedor. Eso me exigirá introducir librerías. Si además me atrevo a crear código usando Jupyther surgió como una evolución de iPython.  

## Documentación:
Básicos:
   * a [api oficial](https://api.mongodb.com/python/current/tutorial.html)
   * [Jupyter en VSC](https://code.visualstudio.com/docs/python/jupyter-support)
   * [docu oficial Mongo](https://docs.python.org/3/)
   * [docker compose](https://docs.docker.com/compose/gettingstarted/)
   * [Jupyter](https://jupyter.org/install) basado en [ipython](http://ipython.org/index.html)
    * [Anaconda](https://www.anaconda.com/distribution/)
    
    
Usar Jupyter:
    * [blogthinkbig](https://empresas.blogthinkbig.com/python-para-todos-2-jupyternotebook/)  
    * [readthedocs](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)  
    * [ipython](http://ipython.org/index.html) es la base de Jupyter
    * [optimizar Jupiter para Git](https://towardsdatascience.com/version-control-with-jupyter-notebooks-f096f4d7035a)
    



## Primer intento

No sé realmente cual es el éxito pero de empezar de alguna manera. Hay un artículo que habla de usar Anaconda dentro de un contenedor así que voy directamente a él. Mi idea es bajar un contendero que ya corra anaconda. Conectar a el el VSC y empezar a escribir código.  

Hay un interesante artículo que seguiré.[getting started with anaconda docker](https://medium.com/@patrickmichelberger/getting-started-with-anaconda-docker-b50a2c482139).  En él se remite a un [artículo](https://www.continuum.io/blog/developer-blog/anaconda-and-docker-better-together-reproducible-data-science) cuyo enlace está roto y que yo creo que por esta [pista](https://www.anaconda.com/data-science-project-data-science-project-encapsulation-deployment/) del [blog](https://www.anaconda.com/category/developer-blog/) deduzco que [contnuumio](https://hub.docker.com/u/continuumio/) y [anaconda]() son la misma empresa.  

Hay otros artículos que hay que leer como el que habla de [cómo preparar un contenedor para un analista de datos](https://towardsdatascience.com/how-to-setup-your-jupyterlab-project-environment-74909dade29b) o este otro que habla de como usar [Git de manera ideal para Jupyter](https://towardsdatascience.com/version-control-with-jupyter-notebooks-f096f4d7035a) o este otro sobre el uso del [control de versiones propio](https://towardsdatascience.com/version-control-with-jupyter-notebooks-f096f4d7035a)
o este otro [sobre un soft que no entiendo](https://www.reviewnb.com/?source=post_page---------------------------)

### Me bajo un contenedor listo con anaconda.

```bash
docker pull continuumio/anaconda3
``` 

y ahora tengo dos opciones. Una es correr una terminal para instalar cosas como librerías. Para eso uso `docker run -i -t continuumio/anaconda3 /bin/bash` del que saldré con exit (parando el contenedor) o con Control+P+Q (para que no pare y entrando con *docker container attach id* ojo que es *Control* también en Mac). 
La otra opción es ejecutar en un navegador directamente el contenedor 
`docker run -i -t -p 8889:8889 continuumio/anaconda3 /bin/bash -c “/opt/conda/bin/conda install jupyter -y --quiet && mkdir /opt/notebooks && /opt/conda/bin/jupyter notebook --notebook-dir=/opt/notebooks --ip='*'—-port=8889 --no-browser”` 
y puedo ver en un navegador:
` http://localhost:8889 `

Pero no me funcionó de manera automática. No encontraba los directorios. Sí me funcionó si lo hacía a mano de la siguiente manera:
```bash
docker run -i -t  -p 8889:8889 continuumio/anaconda3 /bin/bash 
```
una vez dentro:
```bash
$ mkdir /opt/notebooks && /opt/conda/bin/conda install jupyter -y --quiet 
$ /opt/conda/bin/jupyter notebook --allow-root --notebook-dir=/opt/notebooks --ip='*' --port=8889 --no-browser
```
Los errores respecto al original eran las comillas, comillas simples, los guiones que a veces eran simples y debían ser dobles o estaban separados cuando debían estar juntos y --allow-root para que pudiera correr como root. Una vez arreglado todo lo intento en una sola línea y...
```bash
$ docker run -it  -p 8889:8889 continuumio/anaconda3 /bin/bash -c "mkdir /opt/notebooks && /opt/conda/bin/conda install jupyter -y --quiet && /opt/conda/bin/jupyter notebook --allow-root --notebook-dir=/opt/notebooks --ip='*' --port=8889 --no-browser "

``` 





### Qué es Dockerfile o un Dockercompose
Dockerfile y Docker-compose son dos ficheros que permiten automatizar la creación de un contenedor. Son complementarios porque cada uno se encarga de algo distinto. La idea es que puedan substituir entre los dos a todas las opciones que usamos en el comando `docker run...` 

#### En el Dockerfile
Se define la imagen inicial de la que se parte, los ficheros locales que vamos a incluir, los directorios que crearemos de nuevo, el software que vas a añadir (por apt-get o pip o conda o ...), los puertos que vamos a abrir, el directorio de trabajo y el primer comando o comandos que se van a ejecutar en cuanto se arranque por primera vez el contenedor.  
El Dockerfile no siempre es necesario si ya tengo una imagen que funciona bien nada más arrancarla como contenedor. Eso significa que si quiero estar seguro de que nada va a fallar puedo contruir una imagen a partir de un contenedor creado con el Dockerfile y olvidarme de este paso.  



#### En el Docker-compose
Indicamos si el contenedor va a correr en *detach* o va a "secuestrar" la terminal o si va a compartir algún volumen con el hospedador o , de tener un puerto abierto, en que puerto escuchará el hospedador ( un redireccionamiento). También se indica si ha de comunicarse con cualquier otro contenedor, cual es su nombre y el alias que tendrá dentro del contenedor que estoy arrancando.  
Esto se puede ver en las líneas de `docker run...` vistas antes. También se pueden ver varios ejemplos en `/Users/luissanchez/Documents/GitLabb/nuevoabra/guionessh/documentacion/leemeDocker.md`    
El formato del fichero de Docker compose es *.yml* y es una especie de JSON pero que no tiene llaves sino que va indexada como en Python. Un ejemplo es este:  

```yml
version: '3'

services:
  database:
    image: predatabase:v1
    restart: unless-stopped
    ports:
    # <Port exposed> : < MySQL Port running inside container>
      - '3306:3306'
```




#### Ejemplo de Dockerfile

```Dockerfile
#Establecemos imagen base
FROM continuumio/anaconda3
#Comando para indicar al contenedor que comando 

RUN mkdir /opt/notebooks \
    && /opt/conda/bin/conda install jupyter -y --quiet 
  

#COPY ./api/ /api/
#ADD /prueba/ .         <-- también vale pero add entiende enlaces y algo más.
#RUN mkdir /popi
#WORKDIR /api 
# aquí no hay ningún volumen a compartir aunque me vendría bien.

EXPOSE 8889

#ejecutar cuando arranque 
ENTRYPOINT /opt/conda/bin/jupyter notebook --allow-root --notebook-dir=/opt/notebooks --ip='*' --port=8889 --no-browser
```

El Dockerfile lo convierto en una imagen con el comando:
`docker image build -t nombreImagen . ` . El punto final es importante. Si el fichero no se llama *Dockerfile* hay que substuir el punto por el nombre del fichero (incluyendo la ruta si es necesario).  
Al lanzar la ejecución diría si quiero compartir un volumen y el redireccionamiento de puertos:  
`$ docker run -p 80:80 --name frontend --link backend:backend abrafront ` 


##### Otros comandos útiles:
* Creo una imagen con ese Dockerfile:  
` $ docker image build -t abraback . `   
*  Guardo la imagen en un fichero:  
` $ docker image save --output abraback01 abraback `   
* Subo la imagen de ese fichero:  
` $ scp  -C  abraback01 luis@10.8.0.121:Documentos/ `    
* Entrego a docker de nube la imagen:  
Entro en el servidor: ` $ ssh luis@10.8.0.121 ` y le entrego la imagen con  
` $ sudo docker image load -i abraback01 `   
* Crear (=correr) el contenedor. Ojo que aquí además enlazo un volumen con link.   
` $ sudo docker run -p 1234:1234 --link backend:backend  --name backend abraback npm start `  
Si el comando fuera muy complejo lo puedo entrecomillar y con el "-c" indico que es un comando. Quedaría en este caso así:  ` $ sudo docker run -p 1234:1234 --link backend:backend  --name backend abraback -c "npm start" `



#### Ejemplos de Docker-compose.yml
Este es el ejemplo del contenedor que usamos para el proyecto DBaul
```yml
version: '3'                  # es la vesrión de docker-compose a usar

services:                     # un servicio y un contenedor Docker deben ser lo mismo.
  database:                   # el nombre del primer servicio/contenedor
    image: predatabase:v1     # uso una imagen en lugar de un Dockerfile
    restart: unless-stopped   # que hacer si se cae el contenedor??
    ports:                    # el redirecc de puertos contenedor:hospedador ??
    # <Port exposed> : < MySQL Port running inside container>
      - '3306:3306'           # ojo al guión
#    volumes:                 # si hubiera volumenes compartidos
#      # Where our data will be persisted
#      - my-db:/var/lib/mysql   # al igual que puertos: contenedor:hospedador ??
  etlpython:       # el nombre del segundo servicio/contenedor
    build:         # uso un Dockerfile en lugar de una imagen 
      context: ..
      dockerfile: .devcontainer/DockerfilePython
    links:         # enlace a la otra imagen que necesita ¿Si tiene otro nombre?
      - "database"
    depends_on:    # antes de este servicio se debe arrancar el otro.
      - database
    volumes:       # volumen compartido. ojo porque como uso VCS hay restricciones
      # Esta linea tiene que coincidir con el workspaceFolder del devcontainer.json para que el VSC encuentre la carpeta de trabajo
      - ..:/workspace
    # Esta linea es necesaria para que no se pare el contenedor al terminar de cargarse
    command: sleep infinity
```  


En mi caso para aprender VSC con Anaconda y SQL y MongoDB usaré:  

```yml
version: '3'                  # es la vesrión de docker-compose a usar

services:                     # un servicio y un contenedor Docker deben ser lo mismo.
  database:                   # el nombre del primer servicio/contenedor
    image: predatabase:v1     # uso una imagen en lugar de un Dockerfile
    restart: unless-stopped   # que hacer si se cae el contenedor??
    ports:                    # el redirecc de puertos contenedor:hospedador ??
    # <Port exposed> : < MySQL Port running inside container>
      - '3306:3306'           # ojo al guión
    volumes:                  # si hubiera volumenes compartidos
      # Where our data will be persisted
      - my-db:/var/lib/mysql
```



## Librerías
Veo que necesito la librería básica de mongo versión Python

In [1]:
from pymongo import MongoClient


ModuleNotFoundError: No module named 'pymongo'